# Chapter 8: 整然形式にデータを再構成

## レシピ
* [変数値カラム名をstackで整然化](#レシピ65-変数値カラム名をstackで整然化)
* [変数値カラム名をmeltで整然化](#レシピ66-変数値カラム名をmeltで整然化)
* [複数の変数グループを同時にスタック](#レシピ67-複数の変数グループを同時にスタック)
* [スタックしたデータを元に戻す](#レシピ68-スタックしたデータを元に戻す)
* [groupby集約の後でunstack](#レシピ69-groupby集約の後でunstack)
* [groupby集約でpivot_tableの代用](#レシピ70-groupby集約でpivot_tableの代用)
* [変形を容易にするレベル軸の名前変更](#レシピ71-変形を容易にするレベル軸の名前変更)
* [複数の変数がカラム名になっている場合の整然化](#レシピ72-複数の変数がカラム名になっている場合の整然化)
* [複数の変数がカラム値の場合の整然化](#レシピ73-複数の変数がカラム値の場合の整然化)
* [複数の値が同じセルにある場合の整然化](#レシピ74-複数の値が同じセルにある場合の整然化)
* [変数がカラム名とカラム値になっている場合の整然化](#レシピ75-変数がカラム名とカラム値になっている場合の整然化)
* [複数の観察が同じテーブルにある場合の整然化](#レシピ76-複数の観察が同じテーブルにある場合の整然化)

In [1]:
import pandas as pd
import numpy as np

# レシピ65 変数値カラム名をstackで整然化

In [3]:
state_fruit = pd.read_csv('data/state_fruit.csv', index_col=0)
state_fruit

,Apple,Orange,Banana
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


In [4]:
# stackメソッドはカラム名をすべて取ってインデックスにくるよう形を変える
state_fruit.stack()

Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [6]:
# reset_indexメソッドで結果をDataFrameにする
state_fruit_tidy = state_fruit.stack().reset_index()
state_fruit_tidy

,level_0,level_1,0
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [7]:
# カラム名に適切な識別子を置き換える
state_fruit_tidy.columns = ['state', 'fruit', 'weight']
state_fruit_tidy

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [8]:
# Seriesのrename_axisメソッドを使い、reset_index前にインデックスレベルの名前を設定
state_fruit.stack().rename_axis(['state', 'fruit'])

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [9]:
# reset_indexメソッドをチェイニングする
state_fruit.stack()\
            .rename_axis(['state', 'fruit'])\
            .reset_index(name='weight')

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [10]:
# stackのコツは変換したくないカラム全てをインデックスにおくこと
state_fruit2 = pd.read_csv('data/state_fruit2.csv')
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [11]:
# インデックスに置かない例
state_fruit2.stack()

0  State       Texas
   Apple          12
   Orange         10
   Banana         40
1  State     Arizona
   Apple           9
   Orange          7
   Banana         12
2  State     Florida
   Apple           0
   Orange         14
   Banana        190
dtype: object

In [12]:
# 上記を正しく変換するにはset_incexで変形されなかったカラムをインデックスにする
state_fruit2.set_index('State').stack()

State          
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

# レシピ66 変数値カラム名をmeltで整然化

In [3]:
# state_fruit2を読み込み、どのカラムを変形し、どのカラムは必要ないか決める
state_fruit2 = pd.read_csv('data/state_fruit2.csv')
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [4]:
# meltメソッドを使う
state_fruit2.melt(id_vars=['State'],
                 value_vars=['Apple', 'Orange', 'Banana'])

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [5]:
# var_nameとvalue_nameで名前を変更できる
state_fruit2.melt(id_vars=['State'],
                 value_vars=['Apple', 'Orange', 'Banana'],
                 var_name='Fruit',
                 value_name='Weight')

,State,Fruit,Weight
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [6]:
# 全ての値を１つのカラム、カラムラベルを別のカラムにする場合
state_fruit2.melt()

,variable,value
0,State,Texas
1,State,Arizona
2,State,Florida
3,Apple,12
4,Apple,9
5,Apple,0
6,Orange,10
7,Orange,7
8,Orange,14
9,Banana,40


In [7]:
# 識別変数だけ指定したい場合
state_fruit2.melt(id_vars='State')

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


# レシピ67 複数の変数グループを同時にスタック

#### wide_to_long関数を使い、俳優名とFacebookのいいねを同時にスタックする

In [8]:
# movieデータセットを読み込む
movie = pd.read_csv('data/movie.csv')
actor = movie[['movie_title', 'actor_1_name',
              'actor_2_name', 'actor_3_name',
              'actor_1_facebook_likes',
              'actor_2_facebook_likes',
              'actor_3_facebook_likes']]
actor.head()

,movie_title,actor_1_name,actor_2_name,actor_3_name,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [9]:
# カラム名を変更する関数を作成する
def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = col_name[:5] + col_name[fb_idx -1:] + col_name[5:fb_idx-1]
    return col_name    

In [12]:
# カラム名を変換する
actor2 = actor.rename(columns=change_col_name)
actor2.head()

,movie_title,actor_1,actor_2,actor_3,actor_facebook_likes_1,actor_facebook_likes_2,actor_facebook_likes_3
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [13]:
# wide_to_long関数を使って、俳優名とFacebookのカラム集合を同時にスタックする
stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = pd.wide_to_long(actor2,
                             stubnames=stubs,
                             i=['movie_title'],
                             j='actor_num',
                             sep='_')
actor2_tidy.head(20)

,,actor,actor_facebook_likes
movie_title,actor_num,,
Avatar,1,CCH Pounder,1000.0
Pirates of the Caribbean: At World's End,1,Johnny Depp,40000.0
Spectre,1,Christoph Waltz,11000.0
The Dark Knight Rises,1,Tom Hardy,27000.0
Star Wars: Episode VII - The Force Awakens,1,Doug Walker,131.0
John Carter,1,Daryl Sabara,640.0
Spider-Man 3,1,J.K. Simmons,24000.0
Tangled,1,Brad Garrett,799.0
Avengers: Age of Ultron,1,Chris Hemsworth,26000.0


In [14]:
df = pd.read_csv('data/stackme.csv')
df

,State,Country,a1,b2,Test,d,e
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [16]:
# a1とb2、dとeとを一緒にスタックするにはカラム名の変更が必要
df2 = df.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2',
                         'd':'group2_a1', 'e':'group2_b2'})
df2

,State,Country,group1_a1,group1_b2,Test,group2_a1,group2_b2
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [17]:
# 正規表現のsuffixパラメータを変えて文字ならなんでもいいようにする
pd.wide_to_long(df2,
               stubnames=['group1', 'group2'],
               i=['State', 'Country', 'Test'],
               j='Label',
               suffix='.+',
               sep='_')

group1  group2
State Country Test  Label                
TX    US      Test1 a1       0.45       2
                    b2       0.30       6
MA    US      Test2 a1       0.03       9
                    b2       1.20       7
ON    CAN     Test3 a1       0.70       4
                    b2       4.20       2

# レシピ68 スタックしたデータを元に戻す

#### データセットをstack/meltとしてすぐにunstack/pivotで逆演算し形を元に戻す

In [18]:
# 学部学生の人種のカラムだけにする
usecol_func = lambda x: 'UGDS_' in x or x == 'INSTNM'
college = pd.read_csv('data/college.csv',
                     index_col='INSTNM',
                     usecols=usecol_func)
college.head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [19]:
# stackメソッドを使って水平カラム名を垂直indexラベルに変換する
college_stacked = college.stack()
college_stacked.head(18)

INSTNM                                         
Alabama A & M University             UGDS_WHITE    0.0333
                                     UGDS_BLACK    0.9353
                                     UGDS_HISP     0.0055
                                     UGDS_ASIAN    0.0019
                                     UGDS_AIAN     0.0024
                                     UGDS_NHPI     0.0019
                                     UGDS_2MOR     0.0000
                                     UGDS_NRA      0.0059
                                     UGDS_UNKN     0.0138
University of Alabama at Birmingham  UGDS_WHITE    0.5922
                                     UGDS_BLACK    0.2600
                                     UGDS_HISP     0.0283
                                     UGDS_ASIAN    0.0518
                                     UGDS_AIAN     0.0022
                                     UGDS_NHPI     0.0007
                                     UGDS_2MOR     0.0368
                        

In [21]:
# undtackメソッドで元に戻す
college_stacked.unstack()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137
...,...,...,...,...,...,...,...,...,...
Hollywood Institute of Beauty Careers-West Palm Beach,0.2182,0.4182,0.2364,0.0182,0.0000,0.0000,0.0000,0.0182,0.0909
Hollywood Institute of Beauty Careers-Casselberry,0.1200,0.3333,0.4400,0.0000,0.0000,0.0000,0.0400,0.0000,0.0667
Coachella Valley Beauty College-Beaumont,0.3284,0.1045,0.4925,0.0149,0.0299,0.0149,0.0149,0.0000,0.0000


In [22]:
# meltとpivotを使って同様なことを行う,
college2 = pd.read_csv('data/college.csv', usecols=usecol_func)
college2.head()

,INSTNM,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
0,Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
1,University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
2,Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
3,University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
4,Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [25]:
college_melted = college2.melt(id_vars='INSTNM',
                              var_name='Race',
                              value_name='Percntage')
college_melted.head()

,INSTNM,Race,Percntage
0,Alabama A & M University,UGDS_WHITE,0.0333
1,University of Alabama at Birmingham,UGDS_WHITE,0.5922
2,Amridge University,UGDS_WHITE,0.2990
3,University of Alabama in Huntsville,UGDS_WHITE,0.6988
4,Alabama State University,UGDS_WHITE,0.0158


In [27]:
# pivotメソッドで元に戻す
melted_inv = college_melted.pivot(index='INSTNM',
                                 columns='Race',
                                 values='Percntage')
melted_inv.head()

Race,UGDS_2MOR,UGDS_AIAN,UGDS_ASIAN,UGDS_BLACK,UGDS_HISP,UGDS_NHPI,UGDS_NRA,UGDS_UNKN,UGDS_WHITE
INSTNM,,,,,,,,,
A & W Healthcare Educators,0.0000,0.0,0.0000,0.9750,0.0250,0.0,0.0000,0.0000,0.0000
A T Still University of Health Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABC Beauty Academy,0.0000,0.0,0.9333,0.0333,0.0333,0.0,0.0000,0.0000,0.0000
ABC Beauty College Inc,0.0000,0.0,0.0000,0.6579,0.0526,0.0,0.0000,0.0000,0.2895
AI Miami International University of Art and Design,0.0018,0.0,0.0018,0.0198,0.4773,0.0,0.0025,0.4644,0.0324


In [29]:
# 順序を復元する為、.locを使って行とカラムを同時に選びindexをリセットする
college2_replication = melted_inv.loc[college2['INSTNM'],college2.columns[1:]].reset_index()
college2.equals(college2_replication)

True

In [31]:
college_stacked = college.stack()
college_stacked.head()

INSTNM                              
Alabama A & M University  UGDS_WHITE    0.0333
                          UGDS_BLACK    0.9353
                          UGDS_HISP     0.0055
                          UGDS_ASIAN    0.0019
                          UGDS_AIAN     0.0024
dtype: float64

In [33]:
# 上記を外側カラムでunstackしたい場合はlevel=0とする
college_stacked.unstack(0)

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,...,MCI Institute of Technology-Boca Raton,West Coast University-Miami,National American University-Houston,Aparicio-Levy Technical College,Fred D. Learey Technical College,Hollywood Institute of Beauty Careers-West Palm Beach,Hollywood Institute of Beauty Careers-Casselberry,Coachella Valley Beauty College-Beaumont,Dewey University-Mayaguez,Coastal Pines Technical College
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,...,0.0199,0.1522,0.1858,0.2431,0.3731,0.2182,0.1200,0.3284,0.0,0.6762
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,...,0.2815,0.1739,0.6443,0.1215,0.1388,0.4182,0.3333,0.1045,0.0,0.2508
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,...,0.6854,0.6087,0.0672,0.6243,0.3080,0.2364,0.4400,0.4925,1.0,0.0359
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,...,0.0132,0.0217,0.0079,0.0055,0.0000,0.0182,0.0000,0.0149,0.0,0.0045
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,...,0.0000,0.0000,0.0079,0.0055,0.0000,0.0000,0.0000,0.0299,0.0,0.0034
UGDS_NHPI,0.0019,0.0007,0.0000,0.0002,0.0006,0.0009,0.0000,0.0010,0.0016,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0149,0.0,0.0017
UGDS_2MOR,0.0000,0.0368,0.0000,0.0172,0.0098,0.0261,0.0000,0.0174,0.0297,0.0000,...,0.0000,0.0435,0.0751,0.0000,0.0022,0.0000,0.0400,0.0149,0.0,0.0191
UGDS_NRA,0.0059,0.0179,0.0000,0.0332,0.0243,0.0268,0.0000,0.0057,0.0397,0.0100,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0182,0.0000,0.0000,0.0,0.0028
UGDS_UNKN,0.0138,0.0100,0.2715,0.0350,0.0137,0.0026,0.0019,0.0334,0.0246,0.0140,...,0.0000,0.0000,0.0119,0.0000,0.1779,0.0909,0.0667,0.0000,0.0,0.0056


In [34]:
# 上記は転置関数を使っても出来る
college.T

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,...,Strayer University-North Dallas,Strayer University-San Antonio,Strayer University-Stafford,WestMed College - Merced,Vantage College,SAE Institute of Technology San Francisco,Rasmussen College - Overland Park,National Personal Training Institute of Cleveland,Bay Area Medical Academy - San Jose Satellite Location,Excel Learning Center-San Antonio South
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NHPI,0.0019,0.0007,0.0000,0.0002,0.0006,0.0009,0.0000,0.0010,0.0016,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_2MOR,0.0000,0.0368,0.0000,0.0172,0.0098,0.0261,0.0000,0.0174,0.0297,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NRA,0.0059,0.0179,0.0000,0.0332,0.0243,0.0268,0.0000,0.0057,0.0397,0.0100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_UNKN,0.0138,0.0100,0.2715,0.0350,0.0137,0.0026,0.0019,0.0334,0.0246,0.0140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
college.transpose()

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,...,Strayer University-North Dallas,Strayer University-San Antonio,Strayer University-Stafford,WestMed College - Merced,Vantage College,SAE Institute of Technology San Francisco,Rasmussen College - Overland Park,National Personal Training Institute of Cleveland,Bay Area Medical Academy - San Jose Satellite Location,Excel Learning Center-San Antonio South
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NHPI,0.0019,0.0007,0.0000,0.0002,0.0006,0.0009,0.0000,0.0010,0.0016,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_2MOR,0.0000,0.0368,0.0000,0.0172,0.0098,0.0261,0.0000,0.0174,0.0297,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NRA,0.0059,0.0179,0.0000,0.0332,0.0243,0.0268,0.0000,0.0057,0.0397,0.0100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_UNKN,0.0138,0.0100,0.2715,0.0350,0.0137,0.0026,0.0019,0.0334,0.0246,0.0140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# レシピ69 groupby集約後でunstack

In [37]:
# 人種ごとに給与の平均を計算する
employee = pd.read_csv('data/employee.csv')
employee.groupby('RACE')['BASE_SALARY'].mean().astype(int)

RACE
American Indian or Alaskan Native    60272
Asian/Pacific Islander               61660
Black or African American            50137
Hispanic/Latino                      52345
Others                               51278
White                                64419
Name: BASE_SALARY, dtype: int32

In [39]:
# 全人種の平均給与を性別別に求める
agg = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'].mean().astype(int)
agg

RACE                               GENDER
American Indian or Alaskan Native  Female    60238
                                   Male      60305
Asian/Pacific Islander             Female    63226
                                   Male      61033
Black or African American          Female    48915
                                   Male      51082
Hispanic/Latino                    Female    46503
                                   Male      54782
Others                             Female    63785
                                   Male      38771
White                              Female    66793
                                   Male      63940
Name: BASE_SALARY, dtype: int32

In [40]:
# 性別をindeexにしてunstackする
agg.unstack('GENDER')

GENDER,Female,Male
RACE,,
American Indian or Alaskan Native,60238,60305
Asian/Pacific Islander,63226,61033
Black or African American,48915,51082
Hispanic/Latino,46503,54782
Others,63785,38771
White,66793,63940


In [41]:
# 人種をindexにてunstackする
agg.unstack('RACE')

RACE,American Indian or Alaskan Native,Asian/Pacific Islander,Black or African American,Hispanic/Latino,Others,White
GENDER,,,,,,
Female,60238,63226,48915,46503,63785,66793
Male,60305,61033,51082,54782,38771,63940


In [42]:
# グループカラム、集約カラムが複数個なら、結果はSeriesではなくDataFrameになる
agg2 = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY']\
                .agg(['mean', 'max', 'min']).astype(int)
agg2

mean     max    min
RACE                              GENDER                      
American Indian or Alaskan Native Female  60238   98536  26125
                                  Male    60305   81239  26125
Asian/Pacific Islander            Female  63226  130416  26125
                                  Male    61033  163228  27914
Black or African American         Female  48915  150416  24960
                                  Male    51082  275000  26125
Hispanic/Latino                   Female  46503  126115  26125
                                  Male    54782  165216  26104
Others                            Female  63785   63785  63785
                                  Male    38771   38771  38771
White                             Female  66793  178331  27955
                                  Male    63940  210588  26125

In [43]:
# GenderカラムをunstackするとMultiIndexカラムになる
agg2.unstack('GENDER')

mean            max            min       
GENDER                            Female   Male  Female    Male Female   Male
RACE                                                                         
American Indian or Alaskan Native  60238  60305   98536   81239  26125  26125
Asian/Pacific Islander             63226  61033  130416  163228  26125  27914
Black or African American          48915  51082  150416  275000  24960  26125
Hispanic/Latino                    46503  54782  126115  165216  26125  26104
Others                             63785  38771   63785   38771  63785  38771
White                              66793  63940  178331  210588  27955  26125

# レシピ70 groupby集約でpivot_tableの代用

#### ピボット表を作り、同じことをgroupby演算を使って行う

In [45]:
# pivot_tableメソッドを使い航空会社と出発飛行場ごとにキャンセル便の総数を求める
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [46]:
fp =flights.pivot_table(index='AIRLINE',
                       columns='ORG_AIR',
                       values='CANCELLED',
                       aggfunc='sum',
                       fill_value=0).round(2)
fp.head()

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0


In [48]:
# groupby集約で同じことを行う
fg = flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()
fg.head()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
Name: CANCELLED, dtype: int64

In [49]:
# unstackメソッドを使い、ORG_AIRをカラムindexにして転置する
fg_unstack = fg.unstack('ORG_AIR', fill_value=0)
fp.equals(fg_unstack)

True

In [51]:
# もっと複雑なピボット表もgroupbyで作成可能
# pivot_tableの場合
flights.pivot_table(index=['AIRLINE', 'MONTH'],
                   columns=['ORG_AIR', 'CANCELLED'],
                   values=['DEP_DELAY', 'DIST'],
                   aggfunc=[np.sum, np.mean],
                   fill_value=0)

sum                                        ...  \
              DEP_DELAY                                        ...   
ORG_AIR             ATL      DEN      DFW     IAH      LAS     ...   
CANCELLED             0  1     0  1     0  1    0  1     0  1  ...   
AIRLINE MONTH                                                  ...   
AA      1           -13  0   113  0  4276 -3  117  0  1036  0  ...   
        2           -39  0    71  0  2662  0    8  0   -55  0  ...   
        3            -2  0    69  0  5692  0  109  0   326  0  ...   
        4             1  0   304  0  3518  0  104  0   790  0  ...   
        5            52  0   352  0  5510  0   55  0    93  0  ...   
...                 ... ..   ... ..   ... ..  ... ..   ... ..  ...   
WN      7          2604  0  1919  0     0  0    0  0  4600  0  ...   
        8          1718  0  1180  0     0  0    0  0  3151  0  ...   
        9          1033  0   705  0     0  0    0  0  1400  0  ...   
        11          700  0  1372  0     0  0    0  0  1309  0  ...   
        12         1679  0  1898  0     0  0    0  0  2327  0  ...   

                      mean                                              \
                      DIST                                               
ORG_AIR                LAX                       MSP               ORD   
CANCELLED                0            1            0    1            0   
AIRLINE MONTH                                                            
AA      1      1678.037037  2475.000000   809.000000  0.0  1068.876033   
        2      1745.892308  1818.000000  1008.000000  0.0  1193.782178   
        3      1781.567568  1744.000000   964.733333  0.0  1058.933333   
        4      1850.923913     0.000000   648.714286  0.0  1094.633094   
        5      1820.478261     0.000000   787.250000  0.0   998.774775   
...                    ...          ...          ...  ...          ...   
WN      7       912.453704   327.777778   647.266667  0.0     0.000000   
        8       835.404040   346.000000   508.703704  0.0     0.000000   
        9       830.210000   317.666667   644.416667  0.0     0.000000   
        11      748.404040   459.333333   573.642857  0.0     0.000000   
        12      811.695238  1067.000000   634.736842  0.0     0.000000   

                                                                    
                                                                    
ORG_AIR                            PHX                 SFO          
CANCELLED               1            0      1            0       1  
AIRLINE MONTH                                                       
AA      1        0.000000  1167.666667    0.0  1860.166667     0.0  
        2      771.142857  1311.461538  868.0  1337.916667  2586.0  
        3      802.000000  1171.363636    0.0  1502.758621     0.0  
        4      943.600000  1266.214286    0.0  1646.903226     0.0  
        5      999.500000  1240.444444    0.0  1436.892857     0.0  
...                   ...          ...    ...          ...     ...  
WN      7        0.000000   799.160256  369.0   636.210526     0.0  
        8        0.000000   891.569767    0.0   644.857143   392.0  
        9        0.000000   872.840000    0.0   731.578947   354.5  
        11       0.000000   823.258741  872.0   580.875000   392.0  
        12       0.000000   778.420690    0.0   782.256410     0.0  

[149 rows x 80 columns]

In [52]:
# groupbyの場合
flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED'])['DEP_DELAY', 'DIST']\
        .agg(['mean', 'sum'])\
        .unstack(['ORG_AIR', 'CANCELLED'], fill_value=0)\
        .swaplevel(0, 1, axis='columns')

mean                                                    \
               DEP_DELAY                                                     
ORG_AIR              ATL            DEN            DFW             IAH       
CANCELLED              0   1          0   1          0    1          0   1   
AIRLINE MONTH                                                                
AA      1      -3.250000 NaN   7.062500 NaN  11.977591 -3.0   9.750000 NaN   
        2      -3.000000 NaN   5.461538 NaN   8.756579  NaN   1.000000 NaN   
        3      -0.166667 NaN   7.666667 NaN  15.383784  NaN  10.900000 NaN   
        4       0.071429 NaN  20.266667 NaN  10.501493  NaN   6.933333 NaN   
        5       5.777778 NaN  23.466667 NaN  16.798780  NaN   3.055556 NaN   
...                  ...  ..        ...  ..        ...  ...        ...  ..   
WN      7      21.700000 NaN  13.143836 NaN        NaN  NaN        NaN NaN   
        8      16.207547 NaN   7.375000 NaN        NaN  NaN        NaN NaN   
        9       8.680672 NaN   4.378882 NaN        NaN  NaN        NaN NaN   
        11      5.932203 NaN   8.215569 NaN        NaN  NaN        NaN NaN   
        12     15.691589 NaN  12.166667 NaN        NaN  NaN        NaN NaN   

                              ...       sum                                 \
                              ...      DIST                                  
ORG_AIR              LAS      ...       LAX              MSP           ORD   
CANCELLED              0   1  ...         0       1        0   1         0   
AIRLINE MONTH                 ...                                            
AA      1      32.375000 NaN  ...  135921.0  2475.0   7281.0 NaN  129334.0   
        2      -3.055556 NaN  ...  113483.0  5454.0   5040.0 NaN  120572.0   
        3      12.074074 NaN  ...  131836.0  1744.0  14471.0 NaN  127072.0   
        4      27.241379 NaN  ...  170285.0     NaN   4541.0 NaN  152154.0   
        5       2.818182 NaN  ...  167484.0     NaN   6298.0 NaN  110864.0   
...                  ...  ..  ...       ...     ...      ...  ..       ...   
WN      7      22.439024 NaN  ...   98545.0  2950.0  19418.0 NaN       NaN   
        8      16.158974 NaN  ...   82705.0  1384.0  13735.0 NaN       NaN   
        9       7.179487 NaN  ...   83021.0   953.0  15466.0 NaN       NaN   
        11      7.522989 NaN  ...   74092.0  1378.0   8031.0 NaN       NaN   
        12     11.994845 NaN  ...   85228.0  2134.0  12060.0 NaN       NaN   

                                                         
                                                         
ORG_AIR                     PHX             SFO          
CANCELLED           1         0      1        0       1  
AIRLINE MONTH                                            
AA      1         NaN   21018.0    NaN  33483.0     NaN  
        2      5398.0   17049.0  868.0  32110.0  2586.0  
        3       802.0   25770.0    NaN  43580.0     NaN  
        4      4718.0   17727.0    NaN  51054.0     NaN  
        5      1999.0   11164.0    NaN  40233.0     NaN  
...               ...       ...    ...      ...     ...  
WN      7         NaN  124669.0  369.0  24176.0     NaN  
        8         NaN  153350.0    NaN  18056.0   784.0  
        9         NaN  130926.0    NaN  27800.0   709.0  
        11        NaN  117726.0  872.0  23235.0   784.0  
        12        NaN  112871.0    NaN  30508.0     NaN  

[149 rows x 80 columns]

# レシピ71 変形を容易にするレベル軸の名前変更

#### 各軸の各レベルに名前を付けて、stack/unstackメソッドを使い、データの形式を変える

In [57]:
# 州立か宗教系かどうかというグループごとに学部学生数とSAT数学点の要約統計量を求める
college = pd.read_csv('data/college.csv')
cg = college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATMTMID']\
            .agg(['count', 'min', 'max']).head(6)
cg

UGDS                 SATMTMID              
                count    min      max    count    min    max
STABBR RELAFFIL                                             
AK     0            7  109.0  12865.0        0    NaN    NaN
       1            3   27.0    275.0        1  503.0  503.0
AL     0           71   12.0  29851.0       13  420.0  590.0
       1           18   13.0   3033.0        8  400.0  560.0
AR     0           68   18.0  21405.0        9  427.0  565.0
       1           14   20.0   4485.0        7  495.0  600.0

In [61]:
# カラムレベルに名前がないので、rename_axisでレベル名をつける
cg = cg.rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
cg

AGG_COLS         UGDS                 SATMTMID              
AGG_FUNCS       count    min      max    count    min    max
STABBR RELAFFIL                                             
AK     0            7  109.0  12865.0        0    NaN    NaN
       1            3   27.0    275.0        1  503.0  503.0
AL     0           71   12.0  29851.0       13  420.0  590.0
       1           18   13.0   3033.0        8  400.0  560.0
AR     0           68   18.0  21405.0        9  427.0  565.0
       1           14   20.0   4485.0        7  495.0  600.0

In [62]:
# stackメソッドでAGG_FUNCSカラムをインデックスレベルに移す
cg.stack('AGG_FUNCS').head()

AGG_COLS                      UGDS  SATMTMID
STABBR RELAFFIL AGG_FUNCS                   
AK     0        count          7.0       0.0
                min          109.0       NaN
                max        12865.0       NaN
       1        count          3.0       1.0
                min           27.0     503.0

In [64]:
# swaplevelメソッドを使い、レベルを入れ替える
cg.stack('AGG_FUNCS').swaplevel('AGG_FUNCS', 'STABBR', axis='index').head()

,,AGG_COLS,UGDS,SATMTMID
AGG_FUNCS,RELAFFIL,STABBR,,
count,0,AK,7.0,0.0
min,0,AK,109.0,NaN
max,0,AK,12865.0,NaN
count,1,AK,3.0,1.0
min,1,AK,27.0,503.0


In [67]:
# sort_indexで軸レベル名を使ってソートのレベルを決める
cg.stack('AGG_FUNCS')\
  .swaplevel('AGG_FUNCS', 'STABBR', axis='index')\
  .sort_index(level='RELAFFIL', axis='index')\
  .sort_index(level='AGG_COLS', axis='columns').head(6)

AGG_COLS                   SATMTMID     UGDS
AGG_FUNCS RELAFFIL STABBR                   
count     0        AK           0.0      7.0
                   AL          13.0     71.0
                   AR           9.0     68.0
max       0        AK           NaN  12865.0
                   AL         590.0  29851.0
                   AR         565.0  21405.0

In [68]:
cg.stack('AGG_FUNCS').unstack(['RELAFFIL', 'STABBR'])

AGG_COLS      UGDS                                          SATMTMID         \
RELAFFIL         0      1        0       1        0       1        0      1   
STABBR          AK     AK       AL      AL       AR      AR       AK     AK   
AGG_FUNCS                                                                     
count          7.0    3.0     71.0    18.0     68.0    14.0      0.0    1.0   
min          109.0   27.0     12.0    13.0     18.0    20.0      NaN  503.0   
max        12865.0  275.0  29851.0  3033.0  21405.0  4485.0      NaN  503.0   

AGG_COLS                               
RELAFFIL       0      1      0      1  
STABBR        AL     AL     AR     AR  
AGG_FUNCS                              
count       13.0    8.0    9.0    7.0  
min        420.0  400.0  427.0  495.0  
max        590.0  560.0  565.0  600.0

In [69]:
# 全カラムをスタックしてSeriesに戻す
cg.stack(['AGG_FUNCS', 'AGG_COLS']).head(12)


STABBR  RELAFFIL  AGG_FUNCS  AGG_COLS
AK      0         count      UGDS            7.0
                             SATMTMID        0.0
                  min        UGDS          109.0
                  max        UGDS        12865.0
        1         count      UGDS            3.0
                             SATMTMID        1.0
                  min        UGDS           27.0
                             SATMTMID      503.0
                  max        UGDS          275.0
                             SATMTMID      503.0
AL      0         count      UGDS           71.0
                             SATMTMID       13.0
dtype: float64

In [70]:
# カラムレベルが自明でレベルの値を全て削除するにはNoneに設定する
cg.rename_axis([None, None], axis='index')\
  .rename_axis([None, None], axis='columns')

UGDS                 SATMTMID              
     count    min      max    count    min    max
AK 0     7  109.0  12865.0        0    NaN    NaN
   1     3   27.0    275.0        1  503.0  503.0
AL 0    71   12.0  29851.0       13  420.0  590.0
   1    18   13.0   3033.0        8  400.0  560.0
AR 0    68   18.0  21405.0        9  427.0  565.0
   1    14   20.0   4485.0        7  495.0  600.0

# レシピ72 複数の変数がカラム名になっている場合の整然化

#### 変数が連結カラム名になってないか調べデータ形式を変更し、テキストをパースして正しい変数値を抽出する

In [3]:
# 男性のweightliftingを読み込み、変数を調べる
weightlifting = pd.read_csv('data/weightlifting_men.csv')
weightlifting

,Weight Category,M35 35-39,M40 40-44,M45 45-49,M50 50-54,M55 55-59,M60 60-64,M65 65-69,M70 70-74,M75 75-79,M80 80+
0,56,137,130,125,115,102,92,80,67,62,55
1,62,152,145,137,127,112,102,90,75,67,57
2,69,167,160,150,140,125,112,97,82,75,60
3,77,182,172,165,150,135,122,107,90,82,65
4,85,192,182,175,160,142,130,112,95,87,70
5,94,202,192,182,167,150,137,120,100,90,75
6,105,210,200,190,175,157,142,122,102,95,80
7,105+,217,207,197,182,165,150,127,107,100,85


In [4]:
# meltメソッドを使い性別と年齢のカラム名をまとめて１つの垂直カラムにする
wl_melt = weightlifting.melt(id_vars='Weight Category',
                            var_name='sex_age',
                            value_name='Oual Total')
wl_melt.head()

,Weight Category,sex_age,Oual Total
0,56,M35 35-39,137
1,62,M35 35-39,152
2,69,M35 35-39,167
3,77,M35 35-39,182
4,85,M35 35-39,192


In [5]:
# strアクセサのsplitメソッドでカラムを２つに分割する
sex_age = wl_melt['sex_age'].str.split(expand=True)
sex_age.head()

,0,1
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39


In [6]:
# カラム名を変更する
sex_age.columns = ['Sex', 'Age Group']
sex_age.head()

,Sex,Age Group
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39


In [7]:
# strアクセサでSexカラムの先頭文字を選ぶ
sex_age['Sex'] = sex_age['Sex'].str[0]
sex_age.head()

,Sex,Age Group
0,M,35-39
1,M,35-39
2,M,35-39
3,M,35-39
4,M,35-39


In [9]:
# pd.concat関数を使い、整然データセットを作る
wl_cat_total = wl_melt[['Weight Category', 'Oual Total']]
wl_tidy = pd.concat([sex_age, wl_cat_total], axis='columns')
wl_tidy.head()

,Sex,Age Group,Weight Category,Oual Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192


In [11]:
# 上記を次のような方法でもOK
cols = ['Weight Category', 'Oual Total']
sex_age[cols] = wl_melt[cols]
sex_age.head()

,Sex,Age Group,Weight Category,Oual Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192


In [14]:
# splitメソッドは使わず、sex_ageカラムから直接新たなカラムを作る方法

age_group = wl_melt.sex_age.str.extract('(\d{2}[-+](?:\d{2})?)',expand=False)

sex = wl_melt.sex_age.str[0]
new_cols = {'Sex':sex, 'Age Group': age_group}
wl_tidy2 = wl_melt.assign(**new_cols).drop('sex_age', axis='columns')

wl_tidy2.sort_index(axis=1).equals(wl_tidy.sort_index(axis=1))

True

# レシピ73 複数の変数がカラム値の場合の整然化

#### 不適切な構造の変数を含むカラムを変換して整然データを作る

In [16]:
# レストランのinspectionsを読み込む
inspections = pd.read_csv('data/restaurant_inspections.csv',parse_dates=['Date'])
inspections.head(10)

,Name,Date,Info,Value
0,E & E Grill House,2017-08-08,Borough,MANHATTAN
1,E & E Grill House,2017-08-08,Cuisine,American
2,E & E Grill House,2017-08-08,Description,Non-food contact surface improperly constructe...
3,E & E Grill House,2017-08-08,Grade,A
4,E & E Grill House,2017-08-08,Score,9.0
5,PIZZA WAGON,2017-04-12,Borough,BROOKLYN
6,PIZZA WAGON,2017-04-12,Cuisine,Pizza
7,PIZZA WAGON,2017-04-12,Description,"Food contact surface not properly washed, rins..."
8,PIZZA WAGON,2017-04-12,Grade,A
9,PIZZA WAGON,2017-04-12,Score,10.0


In [18]:
# pivotを行う
inspections.pivot(index=['Name', 'Date'],
                 columns='Info', values='Value')

NotImplementedError: > 1 ndim Categorical are not supported at this time

In [19]:
# Name, Date,Infoをインデックスにする
inspections.set_index(['Name', 'Date', 'Info']).head(10)

Value
Name              Date       Info                                                          
E & E Grill House 2017-08-08 Borough                                              MANHATTAN
                             Cuisine                                               American
                             Description  Non-food contact surface improperly constructe...
                             Grade                                                        A
                             Score                                                      9.0
PIZZA WAGON       2017-04-12 Borough                                               BROOKLYN
                             Cuisine                                                  Pizza
                             Description  Food contact surface not properly washed, rins...
                             Grade                                                        A
                             Score                                                     10.0

In [20]:
# unstackメソッドを使いInfoカラムの全値をカラムに変換する
inspections.set_index(['Name', 'Date', 'Info']).unstack('Info').head()

Value  \
Info                                              Borough   
Name                                Date                    
3 STAR JUICE CENTER                 2017-05-10   BROOKLYN   
A & L PIZZA RESTAURANT              2017-08-22   BROOKLYN   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25   BROOKLYN   
ANTOJITOS DELI FOOD                 2017-06-01   BROOKLYN   
BANGIA                              2017-06-16  MANHATTAN   

                                                                                                   \
Info                                                                                      Cuisine   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10                     Juice, Smoothies, Fruit Salads   
A & L PIZZA RESTAURANT              2017-08-22                                              Pizza   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25                                            Turkish   
ANTOJITOS DELI FOOD                 2017-06-01  Latin (Cuban, Dominican, Puerto Rican, South &...   
BANGIA                              2017-06-16                                             Korean   

                                                                                                   \
Info                                                                                  Description   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10  Facility not vermin proof. Harborage or condit...   
A & L PIZZA RESTAURANT              2017-08-22  Facility not vermin proof. Harborage or condit...   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25  Plumbing not properly installed or maintained;...   
ANTOJITOS DELI FOOD                 2017-06-01  Live roaches present in facility's food and/or...   
BANGIA                              2017-06-16  Covered garbage receptacle not provided or ina...   

                                                            
Info                                           Grade Score  
Name                                Date                    
3 STAR JUICE CENTER                 2017-05-10     A  12.0  
A & L PIZZA RESTAURANT              2017-08-22     A   9.0  
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25     A  13.0  
ANTOJITOS DELI FOOD                 2017-06-01     A  10.0  
BANGIA                              2017-06-16     A   9.0

In [30]:
# rest_indexメソッドでインデックスレベルをカラムにする
insp_tidy = inspections.set_index(['Name','Date', 'Info'])\
                        .unstack('Info')\
                        .reset_index(col_level=-1)
insp_tidy.head(10)

Value  \
Info                                 Name       Date    Borough   
0                     3 STAR JUICE CENTER 2017-05-10   BROOKLYN   
1                  A & L PIZZA RESTAURANT 2017-08-22   BROOKLYN   
2     AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25   BROOKLYN   
3                     ANTOJITOS DELI FOOD 2017-06-01   BROOKLYN   
4                                  BANGIA 2017-06-16  MANHATTAN   
5                         BANGKOK CUISINE 2017-07-19  MANHATTAN   
6                                   BASIL 2017-05-03   BROOKLYN   
7                              BEIT JEDDO 2017-03-23   BROOKLYN   
8                  BIG FLEISHIG'S EXPRESS 2017-02-22   BROOKLYN   
9                    BLOSSOM  ON COLUMBUS 2017-01-25  MANHATTAN   

                                                         \
Info                                            Cuisine   
0                        Juice, Smoothies, Fruit Salads   
1                                                 Pizza   
2                                               Turkish   
3     Latin (Cuban, Dominican, Puerto Rican, South &...   
4                                                Korean   
5                                                  Thai   
6                                         Jewish/Kosher   
7                                        Middle Eastern   
8                                         Jewish/Kosher   
9                                              American   

                                                                     
Info                                        Description Grade Score  
0     Facility not vermin proof. Harborage or condit...     A  12.0  
1     Facility not vermin proof. Harborage or condit...     A   9.0  
2     Plumbing not properly installed or maintained;...     A  13.0  
3     Live roaches present in facility's food and/or...     A  10.0  
4     Covered garbage receptacle not provided or ina...     A   9.0  
5     Non-food contact surface improperly constructe...     A  13.0  
6     Cold food item held above 41Âº F (smoked fish ...     A  13.0  
7                          Thawing procedures improper.     A  13.0  
8     Single service item reused, improperly stored,...     A  12.0  
9     Food contact surface not properly washed, rins...     A  10.0

In [27]:
# MultiIndexメソッドdroplevelを使い最上位カラムを削除してindexレベルの名前をNoneでなくす
insp_tidy.columns = insp_tidy.columns.droplevel(0).rename(None)
insp_tidy.head()

,Name,Date,Borough,Cuisine,Description,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
4,BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0


In [28]:
# squeezeメソッドを使い上記と同じ結果を得る
inspections.set_index(['Name','Date', 'Info']) \
          .squeeze() \
          .unstack('Info') \
          .reset_index() \
          .rename_axis(None, axis='columns')

,Name,Date,Borough,Cuisine,Description,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
4,BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0
...,...,...,...,...,...,...,...
95,VALL'S PIZZERIA,2017-03-15,STATEN ISLAND,Pizza/Italian,Wiping cloths soiled or not stored in sanitizi...,A,9.0
96,VIP GRILL,2017-06-12,BROOKLYN,Jewish/Kosher,Hot food item not held at or above 140Âº F.,A,10.0
97,WAHIZZA,2017-04-13,MANHATTAN,Pizza,"No facilities available to wash, rinse and san...",A,10.0
98,WANG MANDOO HOUSE,2017-08-29,QUEENS,Korean,Accurate thermometer not provided in refrigera...,A,12.0


In [32]:
# squeezeメソッドは1カラムのみDataFrameで使用でき、Seriesに変換する
Squeeze_test = inspections.set_index(['Name','Date', 'Info']) \
          .squeeze() 
Squeeze_test

Name                           Date        Info       
E & E Grill House              2017-08-08  Borough                                                MANHATTAN
                                           Cuisine                                                 American
                                           Description    Non-food contact surface improperly constructe...
                                           Grade                                                          A
                                           Score                                                        9.0
                                                                                ...                        
PIER SIXTY ONE-THE LIGHTHOUSE  2017-09-01  Borough                                                MANHATTAN
                                           Cuisine                                                 American
                                           Description    Filth flies or food/ref

In [34]:
# pivot_tableメソッドを使い集約関数(aggfunc)にfristを用いてグループの先頭の値をつかえば、上記と同じ結果になる
inspections.pivot_table(index=['Name', 'Date'], columns='Info',
                       values='Value', aggfunc='first').reset_index().rename_axis(None, axis='columns')

,Name,Date,Borough,Cuisine,Description,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
4,BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0
...,...,...,...,...,...,...,...
95,VALL'S PIZZERIA,2017-03-15,STATEN ISLAND,Pizza/Italian,Wiping cloths soiled or not stored in sanitizi...,A,9.0
96,VIP GRILL,2017-06-12,BROOKLYN,Jewish/Kosher,Hot food item not held at or above 140Âº F.,A,10.0
97,WAHIZZA,2017-04-13,MANHATTAN,Pizza,"No facilities available to wash, rinse and san...",A,10.0
98,WANG MANDOO HOUSE,2017-08-29,QUEENS,Korean,Accurate thermometer not provided in refrigera...,A,12.0


# レシピ74 複数の値が同じセルにある場合の整然化

In [35]:
# テキサス州のcitiesを読み込み、変数を識別する
cities = pd.read_csv('data/texas_cities.csv')
cities

,City,Geolocation
0,Houston,"29.7604° N, 95.3698° W"
1,Dallas,"32.7767° N, 96.7970° W"
2,Austin,"30.2672° N, 97.7431° W"


In [36]:
# カラムを分類する
geolocations = cities.Geolocation.str.split(pat='. ',expand=True)
geolocations.columns = ['latitude', 'latitude direction',
                       'longitude', 'longitude direction']
geolocations

,latitude,latitude direction,longitude,longitude direction
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [37]:
# latitudeとlongitudeをfloatに変換する
geolocations = geolocations.astype({'latitude':'float',
                                   'longitude':'float'})
geolocations.dtypes

latitude               float64
latitude direction      object
longitude              float64
longitude direction     object
dtype: object

In [38]:
# cityカラムに連結する
cities_tidy = pd.concat([cities['City'], geolocations], axis='columns')
cities_tidy

,City,latitude,latitude direction,longitude,longitude direction
0,Houston,29.7604,N,95.3698,W
1,Dallas,32.7767,N,96.7970,W
2,Austin,30.2672,N,97.7431,W


In [39]:
# セル内で特定のグループを抽出するextractメソッド
# キャプチャグループを括弧でくくらないといけない
cities.Geolocation.str.extract('([0-9.]+). (N|S), ([0-9.]+). (E|W)',expand=True)

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


# レシピ75 変数がカラム名とカラム値になっている場合の整然化

In [40]:
# sensorsデータを読み込む
sensors = pd.read_csv('data/sensors.csv')
sensors

,Group,Property,2012,2013,2014,2015,2016
0,A,Pressure,928,873,814,973,870
1,A,Temperature,1026,1038,1009,1036,1042
2,A,Flow,819,806,861,882,856
3,B,Pressure,817,877,914,806,942
4,B,Temperature,1008,1041,1009,1002,1013
5,B,Flow,887,899,837,824,873


In [41]:
# meltメソッドで年をカラムに変換する
sensors.melt(id_vars=['Group', 'Property'], var_name='Year').head(6)

,Group,Property,Year,value
0,A,Pressure,2012,928
1,A,Temperature,2012,1026
2,A,Flow,2012,819
3,B,Pressure,2012,817
4,B,Temperature,2012,1008
5,B,Flow,2012,887


In [42]:
# pivot_tableメソッドを使い、Propertyカラムを新たなカラム名に変換する
sensors.melt(id_vars=['Group', 'Property'], var_name='Year')\
        .pivot_table(index=['Group', 'Year'],
                    columns='Property', values='value')\
        .reset_index()\
        .rename_axis(None, axis='columns')

,Group,Year,Flow,Pressure,Temperature
0,A,2012,819,928,1026
1,A,2013,806,873,1038
2,A,2014,861,814,1009
3,A,2015,882,973,1036
4,A,2016,856,870,1042
5,B,2012,887,817,1008
6,B,2013,899,877,1041
7,B,2014,837,914,1009
8,B,2015,824,806,1002
9,B,2016,873,942,1013


In [43]:
# 上記をstackとundtackで代用できる、コツは変換対象でないカラムをまずindexにすること

sensors.set_index(['Group', 'Property'])\
        .stack()\
        .unstack('Property')\
        .rename_axis(['Group', 'Year'], axis='index')\
        .rename_axis(None, axis='columns')\
        .reset_index()

,Group,Year,Flow,Pressure,Temperature
0,A,2012,819,928,1026
1,A,2013,806,873,1038
2,A,2014,861,814,1009
3,A,2015,882,973,1036
4,A,2016,856,870,1042
5,B,2012,887,817,1008
6,B,2013,899,877,1041
7,B,2014,837,914,1009
8,B,2015,824,806,1002
9,B,2016,873,942,1013


# レシピ75 複数の観察が同じテーブルにある場合の整然化

#### 3つの観測ユニットを見つけ、別々のテーブルを作成する

In [44]:
movie = pd.read_csv('data/movie_altered.csv')
movie.head()

,title,rating,year,duration,director_1,director_fb_likes_1,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
0,Avatar,PG-13,2009.0,178.0,James Cameron,0.0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,PG-13,2007.0,169.0,Gore Verbinski,563.0,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,PG-13,2015.0,148.0,Sam Mendes,0.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,PG-13,2012.0,164.0,Christopher Nolan,22000.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN,Doug Walker,131.0,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [45]:
# insertメソッドを使い、映画に異なる識別番号を振るカラムidを作る
movie.insert(0,'id', np.arange(len(movie)))
movie.head()

,id,title,rating,year,duration,director_1,director_fb_likes_1,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
0,0,Avatar,PG-13,2009.0,178.0,James Cameron,0.0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,1,Pirates of the Caribbean: At World's End,PG-13,2007.0,169.0,Gore Verbinski,563.0,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,2,Spectre,PG-13,2015.0,148.0,Sam Mendes,0.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,3,The Dark Knight Rises,PG-13,2012.0,164.0,Christopher Nolan,22000.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN,Doug Walker,131.0,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [47]:
# wide_to_long関数を使い、整然化する
stubnames = ['director', 'director_fb_likes', 'actor', 'actor_fb_likes']
movie_long = pd.wide_to_long(movie,
                            stubnames=stubnames,
                            i='id',
                            j='num',
                            sep='_').reset_index()
movie_long['num'] = movie_long['num'].astype(int)
movie_long.head(9)

,id,num,year,title,rating,duration,director,director_fb_likes,actor,actor_fb_likes
0,0,1,2009.0,Avatar,PG-13,178.0,James Cameron,0.0,CCH Pounder,1000.0
1,0,2,2009.0,Avatar,PG-13,178.0,NaN,NaN,Joel David Moore,936.0
2,0,3,2009.0,Avatar,PG-13,178.0,NaN,NaN,Wes Studi,855.0
3,1,1,2007.0,Pirates of the Caribbean: At World's End,PG-13,169.0,Gore Verbinski,563.0,Johnny Depp,40000.0
4,1,2,2007.0,Pirates of the Caribbean: At World's End,PG-13,169.0,NaN,NaN,Orlando Bloom,5000.0
5,1,3,2007.0,Pirates of the Caribbean: At World's End,PG-13,169.0,NaN,NaN,Jack Davenport,1000.0
6,2,1,2015.0,Spectre,PG-13,148.0,Sam Mendes,0.0,Christoph Waltz,11000.0
7,2,2,2015.0,Spectre,PG-13,148.0,NaN,NaN,Rory Kinnear,393.0
8,2,3,2015.0,Spectre,PG-13,148.0,NaN,NaN,Stephanie Sigman,161.0


In [51]:
# テーブル分割する
movie_table = movie_long[['id','title', 'year', 'duration', 'rating']]
director_table = movie_long[['id', 'director', 'num', 'director_fb_likes']]
actor_table = movie_long[['id', 'actor', 'num', 'actor_fb_likes']]

In [52]:
movie_table.head(10)

,id,title,year,duration,rating
0,0,Avatar,2009.0,178.0,PG-13
1,0,Avatar,2009.0,178.0,PG-13
2,0,Avatar,2009.0,178.0,PG-13
3,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
4,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
5,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
6,2,Spectre,2015.0,148.0,PG-13
7,2,Spectre,2015.0,148.0,PG-13
8,2,Spectre,2015.0,148.0,PG-13
9,3,The Dark Knight Rises,2012.0,164.0,PG-13


In [54]:
# 重複、欠損データを処置する
movie_table = movie_table.drop_duplicates().reset_index(drop=True)
director_table = director_table.dropna().reset_index(drop=True)
actor_table = actor_table.dropna().reset_index(drop=True)

In [55]:
movie_table.head(10)

,id,title,year,duration,rating
0,0,Avatar,2009.0,178.0,PG-13
1,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
2,2,Spectre,2015.0,148.0,PG-13
3,3,The Dark Knight Rises,2012.0,164.0,PG-13
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN
5,5,John Carter,2012.0,132.0,PG-13
6,6,Spider-Man 3,2007.0,156.0,PG-13
7,7,Tangled,2010.0,100.0,PG
8,8,Avengers: Age of Ultron,2015.0,141.0,PG-13
9,9,Harry Potter and the Half-Blood Prince,2009.0,153.0,PG


In [56]:
# 元のデータセットと作成したデータセットのメモリ使用量を比較
movie.memory_usage(deep=True).sum()

2289818

In [57]:
movie_table.memory_usage(deep=True).sum() + \
director_table.memory_usage(deep=True).sum() + \
actor_table.memory_usage(deep=True).sum()

2538166

In [60]:
# 俳優と監督のテーブルにidカラムを作成する
director_cat = pd.Categorical(director_table['director'])
director_table.insert(1, 'director_id', director_cat.codes)
actor_cat = pd.Categorical(actor_table['actor'])
actor_table.insert(1, 'actor_id', actor_cat.codes)

In [61]:
actor_table.head()

,id,actor_id,actor,num,actor_fb_likes
0,0,824,CCH Pounder,1,1000.0
1,0,2867,Joel David Moore,2,936.0
2,0,6099,Wes Studi,3,855.0
3,1,2971,Johnny Depp,1,40000.0
4,1,4536,Orlando Bloom,2,5000.0


In [62]:
director_table.head()

,id,director_id,director,num,director_fb_likes
0,0,922,James Cameron,1,0.0
1,1,794,Gore Verbinski,1,563.0
2,2,2020,Sam Mendes,1,0.0
3,3,373,Christopher Nolan,1,22000.0
4,4,600,Doug Walker,1,131.0


In [63]:
# directorテーブルの作成
director_associative = director_table[['id', 'director_id', 'num']]
dcols = ['director_id', 'director', 'director_fb_likes']
director_unique = director_table[dcols].drop_duplicates().reset_index(drop=True)

director_unique.head()

,director_id,director,director_fb_likes
0,922,James Cameron,0.0
1,794,Gore Verbinski,563.0
2,2020,Sam Mendes,0.0
3,373,Christopher Nolan,22000.0
4,600,Doug Walker,131.0


In [64]:
# actorテーブルの作成
actor_associative = actor_table[['id', 'actor_id', 'num']]
acols = ['actor_id', 'actor', 'actor_fb_likes']
actor_unique = actor_table[acols].drop_duplicates().reset_index(drop=True)

actor_associative.head()

,id,actor_id,num
0,0,824,1
1,0,2867,2
2,0,6099,3
3,1,2971,1
4,1,4536,2


In [65]:
# メモリ使用量を再計算
movie_table.memory_usage(deep=True).sum() + \
director_associative.memory_usage(deep=True).sum() + \
director_unique.memory_usage(deep=True).sum() + \
actor_associative.memory_usage(deep=True).sum() + \
actor_unique.memory_usage(deep=True).sum()

1746766